In [3]:
import pygame
import math
import numpy as np
import sys

pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

# 정육면체 꼭짓점 정의
cube_vertex = np.array([
    [1, 1, 1],
    [1, 1, -1],
    [1, -1, 1],
    [1, -1, -1],
    [-1, 1, 1],
    [-1, 1, -1],
    [-1, -1, 1],
    [-1, -1, -1]
])

# 정육면체의 간선 정의
edges = [
    (0, 1), (0, 2), (0, 4),
    (1, 3), (1, 5),
    (2, 3), (2, 6),
    (3, 7),
    (4, 5), (4, 6),
    (5, 7),
    (6, 7)
]

# 카메라 설정
camera_pos = np.array([0.0, 0.0, -5.0])
camera_target = np.array([0.0, 0.0, 0.0])
camera_up = np.array([0.0, 1.0, 0.0])

# look-at 뷰 행렬 계산 함수
def look_at(camera_pos, camera_target, camera_up):
    forward = camera_target - camera_pos
    forward /= np.linalg.norm(forward)

    right = np.cross(camera_up, forward)
    right /= np.linalg.norm(right)

    up = np.cross(forward, right)
    mat = np.array([right, up, forward])
    return mat

# 3D → 2D 투영 함수
def project(point3d):
    z = point3d[2] + 5  # z는 원근 효과 조절
    if z == 0: z = 0.001  # 0으로 나누는 오류 방지
    factor = 500 / z
    x = point3d[0] * factor + width // 2
    y = -point3d[1] * factor + height // 2
    return (int(x), int(y))

# 회전 행렬
def x_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])

def y_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, 0, sin_theta],
        [0, 1, 0],
        [-sin_theta, 0, cos_theta]
    ])

def z_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])

angle_x = angle_y = 0.0  # Z축 회전은 생략

# 메인 루프
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    # 마우스 위치에 따라 회전 각도 계산
    mx, my = pygame.mouse.get_pos()
    angle_y = (mx - width // 2) * 0.005
    angle_x = (my - height // 2) * 0.005

    # 회전 행렬 생성
    rotation = x_rotate(angle_x) @ y_rotate(angle_y)

    # 카메라 변환 행렬
    view_matrix = look_at(camera_pos, camera_target, camera_up)

    # 꼭짓점 변환 (회전 + 뷰 + 투영)
    transformed_vertex = []
    for v in cube_vertex:
        rotated = rotation @ v
        camera_space = view_matrix @ (rotated - camera_pos)
        projected = project(camera_space)
        transformed_vertex.append(projected)

    # 화면 초기화
    screen.fill((0, 0, 0))

    # 간선 그리기
    for edge in edges:
        pygame.draw.line(screen, (255, 255, 255),
                         transformed_vertex[edge[0]],
                         transformed_vertex[edge[1]], 2)

    pygame.display.flip()
    clock.tick(60)


SystemExit: 